PDT-5 hahaha

In [2]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)
db = client['PDT']
tweets = db.tweets2
hashtags = db.tweets
authors = db.authors

In [4]:
df = pd.DataFrame(list(tweets.aggregate([{
       '$lookup' : {'from': 'authors','localField': 'author_id','foreignField': 'pgID','as': 'author' }
}])))

In [5]:
df['hashtags'] = list(hashtags.find({}, {'_id': 0, 'hashtags': 1}))

In [6]:
df['author'] = df['author'].apply(lambda x: x[0])

In [7]:
df['hashtags'] = df['hashtags'].apply(lambda x: x['hashtags'])

In [8]:
df['hashtags'] = df['hashtags'].apply(lambda x: [i['value'] for i in x])

In [9]:
df

,_id,id,content,location,retweet_count,favorite_count,compound,neu,neg,pos,happened_at,author_id,country_id,parent_id,hashtags,author
0,61b620c45f1cf5a0efe3ccf1,1261437291278397444,Evolution of a #Qanon \r\n\r\nHandle with care...,None,16,0,0.6249,0.541,0.000,0.459,2020-05-15 21:24:21,790231392810524673,NaN,None,"[MAGA, KAG]","{'_id': 619e61b7083b1ddeaa7461dd, 'pgID': '790..."
1,61b620c45f1cf5a0efe3ccf2,1220676334227218433,RT @Jordan_Sather_: I didn't say a word about ...,None,596,0,-0.6310,0.835,0.165,0.000,2020-01-24 11:54:51,3403240360,NaN,1220491726214717440,"[MAGA, Trump, scleg, nhleg, vtleg, flleg, vale...","{'_id': 619e61b7083b1ddeaa7461b0, 'pgID': '340..."
2,61b620c45f1cf5a0efe3ccf3,1224225264533508096,RT @AnBeOnd: Oh Lord... This should make you s...,None,163,0,-0.5106,0.784,0.216,0.000,2020-02-03 06:57:02,940326182016258048,NaN,1224189632142888967,"[MAGA, Trump, ncpol, scpol, NRA, ncprimary, nc...","{'_id': 619e61b7083b1ddeaa746063, 'pgID': '940..."
3,61b620c45f1cf5a0efe3ccf4,1226576928208867328,BREAKING\n\nWashington: Doctors SUCCESSFULLY T...,None,34,31,0.8668,0.590,0.000,0.410,2020-02-09 18:41:43,767424168669229057,NaN,None,"[QAnon, ThesePeop, PedoGate, GreatAwakenin, Mi...","{'_id': 619e61b7083b1ddeaa746064, 'pgID': '767..."
4,61b620c45f1cf5a0efe3ccf5,1232372878764462081,Yesterday Biden claimed he negotiated with Den...,None,4,5,-0.6486,0.858,0.142,0.000,2020-02-25 18:32:45,767424168669229057,NaN,None,"[QAnon, WWG1WGA]","{'_id': 619e61b7083b1ddeaa746064, 'pgID': '767..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680181,61b620c55f1cf5a0efee2de6,1266283626087374848,Hydroxychloroquine+\r\nQ\r\nDEM PANIC!\r\n#COV...,None,11372,0,-0.6514,0.480,0.520,0.000,2020-05-29 06:21:57,814219806077427712,NaN,None,[QAnon],"{'_id': 619e61b8083b1ddeaa7ebcae, 'pgID': '814..."
680182,61b620c55f1cf5a0efee2de7,1266309033033687041,Barr just dropped the bomb!! Nailed it!!\r\n\r...,None,2987,0,0.6776,0.674,0.077,0.249,2020-05-29 08:02:55,42138264,NaN,None,"[WWG1WGA, QAnon, QArmy, TheGreatAwakening]","{'_id': 619e61b8083b1ddeaa7ebea7, 'pgID': '421..."
680183,61b620c55f1cf5a0efee2de8,1266309147689136128,BOOM! George Floyd's killer isn't a cop. He is...,None,2749,0,-0.7142,0.866,0.134,0.000,2020-05-29 08:03:22,1165342293567725573,NaN,None,[QAnon],"{'_id': 619e61b8083b1ddeaa7ebcb8, 'pgID': '116..."
680184,61b620c55f1cf5a0efee2de9,1266369696758022153,Barr just dropped the bomb!! Nailed it!!\r\n\r...,None,2984,0,0.6776,0.674,0.077,0.249,2020-05-29 12:03:58,954803317003005952,NaN,None,"[MAGA, AmericaFirst, Dobbs]","{'_id': 619e61b8083b1ddeaa7ebc93, 'pgID': '954..."


In [10]:
def modify(x):
    x['id'] = x.pop('pgID')
    x['follower_count'] = x.pop('folCount')
    x['friends_count'] = x.pop('friendCount')
    x['statuses_count'] = x.pop('statCount')
    return x

In [11]:
df['author'].apply(lambda x: modify(x))

0         {'_id': 619e61b7083b1ddeaa7461dd, 'screen_name...
1         {'_id': 619e61b7083b1ddeaa7461b0, 'screen_name...
2         {'_id': 619e61b7083b1ddeaa746063, 'screen_name...
3         {'_id': 619e61b7083b1ddeaa746064, 'screen_name...
4         {'_id': 619e61b7083b1ddeaa746064, 'screen_name...
                                ...                        
680181    {'_id': 619e61b8083b1ddeaa7ebcae, 'screen_name...
680182    {'_id': 619e61b8083b1ddeaa7ebea7, 'screen_name...
680183    {'_id': 619e61b8083b1ddeaa7ebcb8, 'screen_name...
680184    {'_id': 619e61b8083b1ddeaa7ebc93, 'screen_name...
680185    {'_id': 619e61b8083b1ddeaa7ebeaf, 'screen_name...
Name: author, Length: 680186, dtype: object

In [12]:
df['author'][0]

{'_id': ObjectId('619e61b7083b1ddeaa7461dd'),
 'screen_name': 'grammyk3boys1gi',
 'name': 'GrammyKwokeup & @AwakenedbyGod #FreeAssange/Flynn',
 'description': '',
 'id': '790231392810524673',
 'follower_count': 2290,
 'friends_count': 2862,
 'statuses_count': 13708}

In [13]:
es=Elasticsearch()

In [14]:
request_body = {
   "settings":{
      "index":{
         "number_of_shards":3,
         "number_of_replicas":2,
         "max_ngram_diff": 10,
         "analysis":{
            "analyzer":{
               "englando":{
                  "tokenizer":"standard",
                  "filter":[
                     "english_possessive_stemmer",
                     "lowercase",
                     "english_stop",
                     "english_stemmer"
                  ],
                  "char_filter":[
                     "html_strip"
                  ]
               },
               "custom_ngram":{
                  "tokenizer":"standard",
                  "filter":[
                     "lowercase",
                     "asciifolding",
                     "filter_ngrams"
                  ],
                  "char_filter":[
                     "html_strip"
                  ]
               },
               "custom_shingles":{
                  "tokenizer":"standard",
                  "filter":[
                     "lowercase",
                     "asciifolding",
                     "filter_shingles"
                  ],
                  "char_filter":[
                     "html_strip"
                  ]
               }
            },
            "filter":{
               "english_stop":{
                  "type":"stop",
                  "stopwords":"_english_"
               },
               "english_stemmer":{
                  "type":"stemmer",
                  "language":"english"
               },
               "english_possessive_stemmer":{
                  "type":"stemmer",
                  "language":"possessive_english"
               },
               "filter_ngrams":{
                  "type":"ngram",
                  "min_gram":1,
                  "max_gram":10
               },
               "filter_shingles":{
                  "type":"shingle",
                  "token_separator":""
               }
            }
         }
      }
   },
   "mappings":{
      "properties":{
         "id":{
            "type":"text"
         },
         "content":{
            "type":"text",
            "analyzer":"englando"
         },
         "retweet_count":{
            "type":"integer"
         },
         "favorite_count":{
            "type":"integer"
         },
         "happened_at":{
            "type":"date",
            "format": "yyyy-MM-dd'T'HH:mm:ss"
         },
         "country_id":{
            "type":"integer",
             "null_value": -1
         },
         "parent_id":{
            "type":"text"
         },
         "pos":{
            "type":"double",
             "null_value": -1
         },
         "neu":{
            "type":"double",
             "null_value": -1
         },
         "neg":{
            "type":"double",
             "null_value": -1
         },
         "compound":{
            "type":"double",
             "null_value": -1
         },
         "author":{
            "type":"nested",
            "properties":{
               "id":{
                  "type":"long"
               },
               "screen_name":{
                  "type":"text",
                  "fields":{
                     "ngram":{
                        "type":"text",
                        "analyzer":"custom_ngram"
                     }
                  }
               },
               "name":{
                  "type":"text",
                  "fields":{
                     "ngram":{
                        "type":"text",
                        "analyzer":"custom_ngram"
                     },
                     "shingles":{
                        "type":"text",
                        "analyzer":"custom_shingles"
                     }
                  }
               },
               "description":{
                  "type":"text",
                  "analyzer":"englando",
                  "fields":{
                     "shingles":{
                        "type":"text",
                        "analyzer":"custom_shingles"
                     }
                  }
               },
               "followers_count":{
                  "type":"integer"
               },
               "friends_count":{
                  "type":"integer"
               },
               "statuses_count":{
                  "type":"integer"
               }
            }
         },
         "hashtags":{
            "type":"keyword",
            "normalizer":"lowercase"
         }
      }
   }
}

In [15]:
es.indices.delete(index='normalized', ignore=[400, 404])
es.indices.create(index='normalized', settings=request_body["settings"], mappings=request_body["mappings"])

C:\Users\JurajBedej\anaconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'normalized'}

In [19]:
bigbulk = []

for index, row in enumerate(df.to_dict(orient='records')):
    if index == 5000:
        break
    row.pop('_id', None)
    row['author'].pop('_id', None)
    doc = row
    doc['_index'] = 'normalized'
    if(str(doc['country_id']).lower() == "NaN".lower()):
        doc['country_id'] = -99
    bigbulk.append(doc)

In [20]:
bigbulk

[{'id': '1261437291278397444',
  'content': 'Evolution of a #Qanon \r\n\r\nHandle with care ;) https://t.co/yjrpWED0uB',
  'location': None,
  'retweet_count': 16,
  'favorite_count': 0,
  'compound': 0.6249,
  'neu': 0.541,
  'neg': 0.0,
  'pos': 0.459,
  'happened_at': Timestamp('2020-05-15 21:24:21'),
  'author_id': '790231392810524673',
  'country_id': -99,
  'parent_id': None,
  'hashtags': ['MAGA', 'KAG'],
  'author': {'screen_name': 'grammyk3boys1gi',
   'name': 'GrammyKwokeup & @AwakenedbyGod #FreeAssange/Flynn',
   'description': '',
   'id': '790231392810524673',
   'follower_count': 2290,
   'friends_count': 2862,
   'statuses_count': 13708},
  '_index': 'normalized'},
 {'id': '1220676334227218433',
  'content': "RT @Jordan_Sather_: I didn't say a word about #QAnon in my tweets re: the coronavirus and vaccines, but nevertheless fake news Buzzfeed use…",
  'location': None,
  'retweet_count': 596,
  'favorite_count': 0,
  'compound': -0.631,
  'neu': 0.835,
  'neg': 0.165,
  

In [18]:
bulk(es, bigbulk)

SerializationError: ({'id': '1261437291278397444', 'content': 'Evolution of a #Qanon \r\n\r\nHandle with care ;) https://t.co/yjrpWED0uB', 'location': None, 'retweet_count': 16, 'favorite_count': 0, 'compound': 0.6249, 'neu': 0.541, 'neg': 0.0, 'pos': 0.459, 'happened_at': Timestamp('2020-05-15 21:24:21'), 'author_id': '790231392810524673', 'country_id': -99, 'parent_id': None, 'hashtags': ['MAGA', 'KAG'], 'author': {'_id': ObjectId('619e61b7083b1ddeaa7461dd'), 'screen_name': 'grammyk3boys1gi', 'name': 'GrammyKwokeup & @AwakenedbyGod #FreeAssange/Flynn', 'description': '', 'id': '790231392810524673', 'follower_count': 2290, 'friends_count': 2862, 'statuses_count': 13708}}, TypeError("Unable to serialize ObjectId('619e61b7083b1ddeaa7461dd') (type: <class 'bson.objectid.ObjectId'>)"))

In [21]:
def modifyRetweetCount(id):
    es.indices.refresh(index='normalized')
    es.update(
        index='normalized', id = id,
        body={"doc": { "retweet_count": 80085 }}
    )

In [22]:
modifyRetweetCount("jqO8r30BIgFuB6tls0P0")

<ipython-input-21-edd8fa68d3f7>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'update' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.update(


NotFoundError: NotFoundError(404, 'document_missing_exception', '[_doc][jqO8r30BIgFuB6tls0P0]: document missing')

In [ ]:
es.search(index='normalized', query = {
    "match": {
        "id": "1246183844815888385"
    }
})

In [27]:
request_body_no_replicas = {
   "settings":{
      "index":{
         "number_of_shards":3,
         "number_of_replicas":0,
         "max_ngram_diff": 10,
         "analysis":{
            "analyzer":{
               "englando":{
                  "tokenizer":"standard",
                  "filter":[
                     "english_possessive_stemmer",
                     "lowercase",
                     "english_stop",
                     "english_stemmer"
                  ],
                  "char_filter":[
                     "html_strip"
                  ]
               },
               "custom_ngram":{
                  "tokenizer":"standard",
                  "filter":[
                     "lowercase",
                     "asciifolding",
                     "filter_ngrams"
                  ],
                  "char_filter":[
                     "html_strip"
                  ]
               },
               "custom_shingles":{
                  "tokenizer":"standard",
                  "filter":[
                     "lowercase",
                     "asciifolding",
                     "filter_shingles"
                  ],
                  "char_filter":[
                     "html_strip"
                  ]
               }
            },
            "filter":{
               "english_stop":{
                  "type":"stop",
                  "stopwords":"_english_"
               },
               "english_stemmer":{
                  "type":"stemmer",
                  "language":"english"
               },
               "english_possessive_stemmer":{
                  "type":"stemmer",
                  "language":"possessive_english"
               },
               "filter_ngrams":{
                  "type":"ngram",
                  "min_gram":1,
                  "max_gram":10
               },
               "filter_shingles":{
                  "type":"shingle",
                  "token_separator":""
               }
            }
         }
      }
   },
   "mappings":{
      "properties":{
         "id":{
            "type":"text"
         },
         "content":{
            "type":"text",
            "analyzer":"englando"
         },
         "retweet_count":{
            "type":"integer"
         },
         "favorite_count":{
            "type":"integer"
         },
         "happened_at":{
            "type":"date",
            "format": "yyyy-MM-dd'T'HH:mm:ss"
         },
         "country_id":{
            "type":"integer",
             "null_value": -1
         },
         "parent_id":{
            "type":"text"
         },
         "pos":{
            "type":"double",
             "null_value": -1
         },
         "neu":{
            "type":"double",
             "null_value": -1
         },
         "neg":{
            "type":"double",
             "null_value": -1
         },
         "compound":{
            "type":"double",
             "null_value": -1
         },
         "author":{
            "type":"nested",
            "properties":{
               "id":{
                  "type":"long"
               },
               "screen_name":{
                  "type":"text",
                  "fields":{
                     "ngram":{
                        "type":"text",
                        "analyzer":"custom_ngram"
                     }
                  }
               },
               "name":{
                  "type":"text",
                  "fields":{
                     "ngram":{
                        "type":"text",
                        "analyzer":"custom_ngram"
                     },
                     "shingles":{
                        "type":"text",
                        "analyzer":"custom_shingles"
                     }
                  }
               },
               "description":{
                  "type":"text",
                  "analyzer":"englando",
                  "fields":{
                     "shingles":{
                        "type":"text",
                        "analyzer":"custom_shingles"
                     }
                  }
               },
               "followers_count":{
                  "type":"integer"
               },
               "friends_count":{
                  "type":"integer"
               },
               "statuses_count":{
                  "type":"integer"
               }
            }
         },
         "hashtags":{
            "type":"keyword",
            "normalizer":"lowercase"
         }
      }
   }
}

In [28]:
es.indices.delete(index='normalized', ignore=[400, 404])
es.indices.create(index='normalized', settings=request_body_no_replicas["settings"], mappings=request_body["mappings"])

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'normalized'}

In [29]:
bigbulk = []

for index, row in enumerate(df.to_dict(orient='records')):
    row['author'].pop('_id', None)
    row.pop('_id', None)
    doc = row
    doc['_index'] = 'normalized'
    if(str(doc['country_id']).lower() == "NaN".lower()):
        doc['country_id'] = -99
    bigbulk.append(doc)
    if(index % 20000 == 0):
        bulk(es, bigbulk)
        bigbulk = []

bulk(es, bigbulk)

(185, [])

In [139]:
es.search(index ="normalized", query = {
    "function_score": {
        "query": {
            "bool": {
                "must": [
                    {
                        "bool": {
                            "should": [
                                {
                                    "match": {
                                        "content": {
                                            "query": "gates s0ros vaccine micr0chip",
                                            "fuzziness": "AUTO",
                                            "analyzer": "englando",
                                            "boost": 8.0
                                        }
                                    }
                                },
                                {
                                    "match": {
                                        "author.name": {
                                            "query": "gates s0ros vaccine micr0chip",
                                            "fuzziness": "AUTO",
                                            "analyzer": "custom_shingles",
                                            "boost": 6.0
                                        }
                                    }
                                },
                                {
                                    "match": {
                                        "author.screen_name": {
                                            "query": "gates s0ros vaccine micr0chip",
                                            "fuzziness": "AUTO",
                                            "analyzer": "custom_ngram",
                                            "boost": 10.0
                                        }
                                    }
                                },
                                {
                                    "match": {
                                        "author.description": {
                                            "query": "gates s0ros vaccine micr0chip",
                                            "fuzziness": "AUTO",
                                            "analyzer": "custom_shingles",
                                            "boost": 6.0
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "filter": [
                    {
                        "nested": {
                            "path": "author",
                            "query": {
                                "range": {
                                    "author.statuses_count": {
                                        "gt": 1000
                                    }
                                }
                            }
                        }
                    },
                    {
                        "term": {
                            "hashtags": "qanon"
                        }
                    }
                ]
            }
        },
        "functions": [
            {
                "filter": {
                    "range": {
                        "retweet_count": {
                            "gte": 100,
                            "lte": 500
                        }
                    }
                },
                "weight": 6
            },
            {
                "filter": {
                    "range": {
                        "author.followers_count": {
                            "gt": 100
                        }
                    }
                },
                "weight": 3
            }
        ]
    }
}, aggs = {
         "hashtags":{
        "terms":{
            "field":"hashtags",
            "size": 100000
        }
    }   
})


{'took': 92,
 'timed_out': False,
 '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 523.1282,
  'hits': [{'_index': 'normalized',
    '_type': '_doc',
    '_id': 'GLHtr30BIgFuB6tlYBJc',
    '_score': 523.1282,
    '_source': {'id': '1258631198642851840',
     'content': 'Gates Is Part Of A Covert Medical Cabal. Their Most Effective Tools For Human Depopulation?\r\n\r\nVaccines Spiked With Sterilization Chemical And Genetically Engineered Viruses With High Mortality Rate, Engineered To Target Specific Races And Genetic Profiles.\r\n\r\n#QAnon #Vaccine @potus https://t.co/H0QtpJDTq0',
     'location': None,
     'retweet_count': 146,
     'favorite_count': 0,
     'compound': 0.5256,
     'neu': 0.914,
     'neg': 0.0,
     'pos': 0.086,
     'happened_at': '2020-05-08T03:33:56',
     'author_id': '824611782673129472',
     'country_id': -99,
     'parent_id': None,
     'hashtags': ['QAnon', 'Grea

In [163]:
tags = [ 'DeepstateVirus',
        'DeepStateFauci',
        'DeepStateVaccine',
        'QAnon',
        'Agenda21',
        'CCPVirus',
        'ClimateChangeHoax',
        'GlobalWarmingHoax',
        'ChinaLiedPeopleDied',
        'SorosVirus',
        '5GCoronavirus',
        'MAGA',
        'WWG1WGA',
        'Chemtrails',
        'flatEarth',
        'MoonLandingHoax',
        'moonhoax',
        'illuminati',
        'pizzaGateIsReal',
        'PedoGateIsReal',
        '911truth',
        '911insidejob',
        'reptilians' ]
tags = ' '.join(tags)
tags

'DeepstateVirus DeepStateFauci DeepStateVaccine QAnon Agenda21 CCPVirus ClimateChangeHoax GlobalWarmingHoax ChinaLiedPeopleDied SorosVirus 5GCoronavirus MAGA WWG1WGA Chemtrails flatEarth MoonLandingHoax moonhoax illuminati pizzaGateIsReal PedoGateIsReal 911truth 911insidejob reptilians'

In [186]:
es.search(index ="normalized", query = {
#     "query": {"hashtags":  tags}
    "match_all": {}
},
aggs = {
    "weeks": {
      "date_histogram": {
        "field": "happened_at",
        "calendar_interval": "week"
      },
        "aggs": {
            "sum_of_retweets": {
              "sum": {
                "field": "retweet_count"
              }
          }
        }
    }
 })

{'took': 128,
 'timed_out': False,
 '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 1.0,
  'hits': [{'_index': 'normalized',
    '_type': '_doc',
    '_id': 'yq_sr30BIgFuB6tlXRqS',
    '_score': 1.0,
    '_source': {'id': '1261437291278397444',
     'content': 'Evolution of a #Qanon \r\n\r\nHandle with care ;) https://t.co/yjrpWED0uB',
     'location': None,
     'retweet_count': 16,
     'favorite_count': 0,
     'compound': 0.6249,
     'neu': 0.541,
     'neg': 0.0,
     'pos': 0.459,
     'happened_at': '2020-05-15T21:24:21',
     'author_id': '790231392810524673',
     'country_id': -99,
     'parent_id': None,
     'hashtags': ['MAGA', 'KAG'],
     'author': {'screen_name': 'grammyk3boys1gi',
      'name': 'GrammyKwokeup & @AwakenedbyGod #FreeAssange/Flynn',
      'description': '',
      'id': '790231392810524673',
      'follower_count': 2290,
      'friends_count': 2862,
      'statuses